<a href="https://colab.research.google.com/github/Sandeep354/customer-churn/blob/main/ANN_Customers_Leaving_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:-1].values # dont require row 0,1,2 so start from 3
y = dataset.iloc[:, -1].values

In [4]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

Label Encoding for binary --> 0 and 1 only

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) # only need to change row of index 2  

In [6]:
print (X) 

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

One Hot Encoding --> values between 0 and 1

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Here, we need index 1 to be one hot encoded
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

X = np.array(ct.fit_transform(X))

In [8]:
print (X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


**Observe that the one hot encoded column comes at index = 0 by default**

### Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

### Feature Scaling
# **Very Important**

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) # not fitting to avoid information overlook/leakage of the test data

## Part 2 - Building the ANN

### Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [15]:
# for loss fn, Binary classification --> binary_crossentropy
# More than 2 classification --> categorical_crossentropy
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [16]:
ann.fit(x=X_train, y=y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 903us/step - loss: 0.5271 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 904us/step - loss: 0.4690 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 896us/step - loss: 0.4491 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 888us/step - loss: 0.4392 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 921us/step - loss: 0.4335 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 875us/step - loss: 0.4287 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 892us/step - loss: 0.4230 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 881us/step - loss: 0.4159 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 937us/step - loss: 0.4087 - accuracy: 0.8181
Epoch 10/100
250/250 [==============================] - 0s 877us/step - l

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [20]:
# converting the above into numeric data (use dummy variable for Geography instead of string)
data_for_pred = [1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]

# IMPORTANT: scale these data
scaled_data_for_pred = sc.transform([data_for_pred]) #sc takes list of dat to pred, here we have only 1 testing data

In [21]:
preds = ann.predict(x=scaled_data_for_pred)
print (preds)

[[0.04129732]]


In [24]:
if preds > 0.5 : print ("Customer will leave")
else: print ("Customer won't leave")

Customer won't leave


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [25]:
y_preds = ann.predict(x=X_test)
y_preds = (y_preds > 0.5) # we dont want pred prob, only 0 or 1 (or False or True)

print (np.concatenate((y_preds.reshape(len(y_preds), 1), y_test.reshape(len(y_test), 1)), axis=1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_preds)
print (cm)
accuracy_score(y_test, y_preds)


[[1511   84]
 [ 194  211]]


0.861